# `run.train`

In [ ]:
#default_exp run.train

run on collie.local
run on collie.local
run on collie.local
run on collie.local
run on collie.local


In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
#export

import os
from pathlib import Path
import sys
import argparse
import time
import shutil
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score

import torch
import torch.optim
from torch.utils.data import DataLoader
from torch.utils.data.sampler import RandomSampler, SequentialSampler, WeightedRandomSampler
from torch.nn import DataParallel
from torch.backends import cudnn
import torch.nn.functional as F
from torch.autograd import Variable

In [ ]:
#export

from kgl_humanprotein.config.config import *
from kgl_humanprotein.utils.common_util import *
from kgl_humanprotein.data_process import generate_meta, create_random_split
from kgl_humanprotein.networks.imageclsnet import init_network
from kgl_humanprotein.datasets.protein_dataset import ProteinDataset
from kgl_humanprotein.utils.augment_util import train_multi_augment2
from kgl_humanprotein.layers.loss import *
from kgl_humanprotein.layers.scheduler import *
from kgl_humanprotein.utils.log_util import Logger

## Combine the subsets' meta data

In [ ]:
#export

def combine_subsets_metadata(dir_data, n_subsets=5):
    df_cells = pd.DataFrame()
    for isubset in range(n_subsets):
        print(f'\rProcessing subset {isubset}...', end='', flush=True)
        pth_feather = (dir_data / 
                       f'humanpro-train-cells-subset{isubset}' /
                       f'humanpro_train_cells_subset{isubset}' / 'train/train.feather')
        
        df = pd.read_feather(pth_feather)
        df['subset'] = isubset
        
        df_cells = df_cells.append(df, ignore_index=True)
    return df_cells

In [ ]:
dir_data = Path('../../kgl_humanprotein_data')
dir_mdata = Path('mdata')
n_subsets = 5

In [ ]:
%%time
df_cells = combine_subsets_metadata(dir_data, n_subsets)

Processing subset 4...CPU times: user 28.5 ms, sys: 31.4 ms, total: 59.9 ms
Wall time: 123 ms


In [ ]:
dir_mdata_raw = dir_mdata/'raw'
dir_mdata_raw.mkdir(exist_ok=True, parents=True)

df_cells.to_feather(dir_mdata_raw/'train.feather')

In [ ]:
del df_cells

## Filter samples

In [ ]:
df_cells = pd.read_feather(dir_mdata_raw/'train.feather')

In [ ]:
# keep single-label samples
df_cells = (df_cells[df_cells['Target'].apply(lambda o: len(o.split('|'))==1)]
            .reset_index(drop=True))

In [ ]:
#export

def cap_number_per_label(df_cells, cap=10_000, idx_start=0):
    df_cells_cap = pd.DataFrame()
    for label in df_cells.Target.unique():
        df = df_cells[df_cells.Target==label]
        if len(df) > cap:
            df = df.iloc[idx_start:idx_start + cap]
        df_cells_cap = df_cells_cap.append(df, ignore_index=True)
    return df_cells_cap

In [ ]:
# Limit number of samples per label

df_cells = cap_number_per_label(df_cells, cap=5_000, idx_start=5_000) 

In [ ]:
df_cells.Target.value_counts()

0     129
10     70
6      52
3      39
15     25
9      17
12     16
5      16
14     15
13     15
Name: Target, dtype: int64

In [ ]:
df_cells[df_cells.Target != '18']

,Id,rle,bbox,Target,max_green,subset
0,0877a7da-bba3-11e8-b2b9-ac1f6b6435d0_0,{'counts': b'^1P5Pk1000000O1O1O01000O01O0001O1...,"[0, 0, 276, 206]",3,160,0
1,0877a7da-bba3-11e8-b2b9-ac1f6b6435d0_1,{'counts': b'PPV:1oo12N1O1O3M2N00001O000000000...,"[163, 0, 438, 325]",3,71,0
2,0877a7da-bba3-11e8-b2b9-ac1f6b6435d0_2,{'counts': b'nTdm0=co11O1O1O000000000O2O000O2N...,"[474, 0, 1250, 207]",3,255,0
3,0877a7da-bba3-11e8-b2b9-ac1f6b6435d0_3,{'counts': b'fUj[21no12N4L1O2N2N2N>B2M3N5K3K4M...,"[1213, 0, 1766, 291]",3,255,0
4,0877a7da-bba3-11e8-b2b9-ac1f6b6435d0_4,{'counts': b'WXTf11no12O1N2N2O1O1N4L3M2O1N2N3L...,"[866, 147, 1494, 509]",3,255,0
...,...,...,...,...,...,...
389,f8d60850-bbbe-11e8-b2ba-ac1f6b6435d0_10,{'counts': b'[`_j02ko16L2N2L7L2N1O2M9H2N1N9H00...,"[423, 1190, 1138, 1802]",14,223,4
390,f8d60850-bbbe-11e8-b2ba-ac1f6b6435d0_11,{'counts': b'V[1\\<dc100O1N2O1O1O1O10000O10000...,"[0, 1214, 390, 1778]",14,255,4
391,f8d60850-bbbe-11e8-b2ba-ac1f6b6435d0_12,{'counts': b'hge?`0_o12N2L6nMV2H2aUNPMkg1R3TXN...,"[250, 1458, 682, 2018]",14,255,4
392,f8d60850-bbbe-11e8-b2ba-ac1f6b6435d0_13,{'counts': b'^ceV1:fo11O1O3M3L2O1O3L3M2N2O1O2N...,"[618, 1642, 1210, 2048]",14,200,4


In [ ]:
df_cells.to_feather(dir_mdata_raw/'train.feather')

## One-hot encode labels

In *bestfitting*'s code, this generates the "meta" files.

In [ ]:
%%time
generate_meta(dir_mdata, 'train.feather')

CPU times: user 103 ms, sys: 4.07 ms, total: 107 ms
Wall time: 113 ms


## Split generation

In [ ]:
%%time
train_meta = pd.read_feather(dir_mdata/'meta'/'train_meta.feather')
create_random_split(dir_mdata, train_meta, n_splits=5, alias='random')
del train_meta

Nucleoplasm 103 26
Nuclear membrane 0 0
Nucleoli 0 0
Nucleoli fibrillar center 31 8
Nuclear speckles 0 0
Nuclear bodies 13 3
Endoplasmic reticulum 42 10
Golgi apparatus 0 0
Intermediate filaments 0 0
Actin filaments 14 3
Microtubules 56 14
Mitotic spindle 0 0
Centrosome 12 4
Plasma membrane 12 3
Mitochondria 12 3
Aggresome 20 5
Cytosol 0 0
Vesicles and punctate cytosolic patterns 0 0
Negative 0 0
create split file: mdata/split/random_folds5/random_train_cv0.feather, shape: (315, 27)
create split file: mdata/split/random_folds5/random_valid_cv0.feather, shape: (79, 27)
create split file: mdata/split/random_folds5/random_train_cv1.feather, shape: (315, 27)
create split file: mdata/split/random_folds5/random_valid_cv1.feather, shape: (79, 27)
create split file: mdata/split/random_folds5/random_train_cv2.feather, shape: (316, 27)
create split file: mdata/split/random_folds5/random_valid_cv2.feather, shape: (78, 27)
create split file: mdata/split/random_folds5/random_train_cv3.feather, shap

## Weighted random sampler

In [ ]:
#export

def get_label_weights(df):
    '''
    Weigh each label with the reciprocal of its probability of
    appearance in the dataset.
    
    Args:
        df (pd.DataFrame): Meta data data-frame.
    '''
    value_counts = df['Target'].value_counts()
    probs = value_counts / value_counts.sum()
    weights = 1 / probs
    return weights.to_dict()

In [ ]:
def test_weighted_sampler():
    dir_data = Path('../../kgl_humanprotein_data/')
    train_split_file = dir_mdata/'split'/'random_folds5'/'random_train_cv0.feather'

    train_dataset = ProteinDataset(dir_data, train_split_file)

    label_weights = get_label_weights(train_dataset.split_df)
    weights = torch.from_numpy(train_dataset.split_df['Target'].apply(lambda o: label_weights[o]).values)
    sampler = WeightedRandomSampler(weights, len(weights))

    train_loader = DataLoader(train_dataset, sampler=sampler)
    
    print(label_weights)
    print(train_dataset.split_df.Target.values)
    print(train_loader.sampler.weights)

In [ ]:
test_weighted_sampler()

{'0': 3.058252427184466, '10': 5.625, '6': 7.5, '3': 10.161290322580646, '15': 15.75, '9': 22.5, '5': 24.23076923076923, '13': 26.249999999999996, '12': 26.249999999999996, '14': 26.249999999999996}
['3' '3' '3' '3' '3' '3' '3' '3' '3' '3' '3' '3' '3' '3' '3' '3' '3' '3'
 '3' '3' '3' '3' '3' '6' '6' '6' '6' '6' '6' '6' '6' '6' '6' '6' '6' '6'
 '6' '6' '6' '6' '6' '6' '6' '6' '6' '6' '6' '6' '6' '6' '6' '6' '6' '6'
 '6' '6' '6' '6' '6' '6' '6' '6' '6' '6' '6' '9' '9' '9' '9' '9' '9' '9'
 '9' '9' '9' '9' '9' '9' '9' '13' '13' '13' '13' '13' '13' '13' '13' '13'
 '13' '13' '13' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0'
 '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0'
 '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '10' '10' '10' '10' '10' '10'
 '10' '10' '10' '10' '10' '10' '10' '10' '10' '10' '10' '10' '10' '10'
 '10' '10' '10' '10' '10' '10' '10' '10' '10' '10' '10' '10' '10' '10'
 '10' '10' '10' '10' '10' '10' '10' '10' '10' '10' '10' '10' '10' '10'
 '

## Run training

In [ ]:
#export

loss_names = ['FocalSymmetricLovaszHardLogLoss']
split_names = ['random_ext_folds5', 
               'random_ext_noleak_clean_folds5']


def main_training(dir_data, dir_mdata, dir_results, out_dir, gpu_id='0', 
                  arch='class_densenet121_dropout', model_multicell=None, 
                  num_classes=19, in_channels=4, loss='FocalSymmetricLovaszHardLogLoss',
                  scheduler='Adam45', epochs=55, img_size=768, crop_size=512, batch_size=32, 
                  workers=3, pin_memory=True, split_name='random_ext_folds5', fold=0, 
                  clipnorm=1, resume=None):
    '''
    PyTorch Protein Classification.  Main training function.
    
    Args:
        dir_data (str, Path): Directory where training subsets are.
        dir_mdata (str, Path): Directory where training meta data is.
        dir_results (std, Path): Directory to save training results.
        out_dir (str): 
            Name/label for this training run.  Will be used to create
            directory under `dir_results`.
        gpu_id (str): GPU id used for training. Default: '0'
        arch (str): Model architecture.  
            Default: ``'class_densenet121_dropout'``
        model_multicell (Path, str): Path to multi-cell model 
            to start training from. Default: None
        num_classes (int): Number of classes. Default: 19 
        in_channels (int): In channels. Default: 4
        loss (str, optional): Loss function. 
            One of ``'FocalSymmetricLovaszHardLogLoss'``. 
            Default: ``'FocalSymmetricLovaszHardLogLoss'``
        scheduler (str): Scheduler name. Default: ``'Adam45'``
        epochs (int): Number of total epochs to run. Default: 55
        img_size (int): Image size.  Default: 768
        crop_size (int): Crop size.  Default: 512
        batch_size (int): Train mini-batch size. Default: 32
        workers (int): Number of data loading workers. Default: 3
        pin_memory (bool): DataLoader's ``pin_memory`` argument.
        split_name (str, optional): Split name.  
            One of: ``'random_ext_folds5'``, 
            or ``'random_ext_noleak_clean_folds5'``. 
            Default: ``'random_ext_folds5'``
        fold (int): Index of fold. Default: 0
        clipnorm (int): Clip grad norm'. Default: 1
        resume (str): Name of the latest checkpoint. Default: None
    '''
    log_out_dir = opj(dir_results, 'logs', out_dir, 'fold%d' % fold)
    if not ope(log_out_dir):
        os.makedirs(log_out_dir)
    log = Logger()
    log.open(opj(log_out_dir, 'log.train.txt'), mode='a')

    model_out_dir = opj(dir_results, 'models', out_dir, 'fold%d' % fold)
    log.write(">> Creating directory if it does not exist:\n>> '{}'\n".format(model_out_dir))
    if not ope(model_out_dir):
        os.makedirs(model_out_dir)

    # set cuda visible device
    os.environ['CUDA_VISIBLE_DEVICES'] = gpu_id
    cudnn.benchmark = True

    # set random seeds
    torch.manual_seed(0)
    torch.cuda.manual_seed_all(0)
    np.random.seed(0)

    model_params = {}
    model_params['architecture'] = arch
    model_params['num_classes'] = num_classes
    model_params['in_channels'] = in_channels
    model = init_network(model_params, model_multicell=model_multicell)
    
    if model_multicell is not None:
        torch.load(model_multicell)

    # move network to gpu
    model = DataParallel(model)
    model.to(DEVICE)

    # define loss function (criterion)
    try:
        criterion = eval(loss)().to(DEVICE)
    except:
        raise(RuntimeError("Loss {} not available!".format(loss)))

    start_epoch = 0
    best_loss = 1e5
    best_epoch = 0
    best_focal = 1e5

    # define scheduler
    try:
        scheduler = eval(scheduler)()
    except:
        raise (RuntimeError("Scheduler {} not available!".format(scheduler)))
    optimizer = scheduler.schedule(model, start_epoch, epochs)[0]

    # optionally resume from a checkpoint
    if resume:
        resume = os.path.join(model_out_dir, resume)
        if os.path.isfile(resume):
            # load checkpoint weights and update model and optimizer
            log.write(">> Loading checkpoint:\n>> '{}'\n".format(resume))

            checkpoint = torch.load(resume)
            start_epoch = checkpoint['epoch']
            best_epoch = checkpoint['best_epoch']
            best_focal = checkpoint['best_score']
            model.module.load_state_dict(checkpoint['state_dict'])

            optimizer_fpath = resume.replace('.pth', '_optim.pth')
            if ope(optimizer_fpath):
                log.write(">> Loading checkpoint:\n>> '{}'\n".format(optimizer_fpath))
                optimizer.load_state_dict(torch.load(optimizer_fpath)['optimizer'])
            log.write(">>>> loaded checkpoint:\n>>>> '{}' (epoch {})\n".format(resume, checkpoint['epoch']))
        else:
            log.write(">> No checkpoint found at '{}'\n".format(resume))

    # Data loading code
    train_transform = train_multi_augment2
    train_split_file = dir_mdata / 'split'/ split_name / f'random_train_cv{fold}.feather'
    train_dataset = ProteinDataset(
        dir_data,
        train_split_file,
        img_size=img_size,
        is_trainset=True,
        return_label=True,
        in_channels=in_channels,
        transform=train_transform,
        crop_size=crop_size,
        random_crop=True)
    
    label_weights = get_label_weights(train_dataset.split_df)
    weights = torch.from_numpy(train_dataset.split_df['Target']
                               .apply(lambda o: label_weights[o]).values)
    sampler = WeightedRandomSampler(weights, len(weights))
    
    train_loader = DataLoader(
        train_dataset,
        sampler=sampler,
        batch_size=batch_size,
        drop_last=True,
        num_workers=workers,
        pin_memory=pin_memory)
    
    valid_split_file = (dir_mdata/'split'/split_name/
                        f'random_valid_cv{fold}.feather')
    
    valid_dataset = ProteinDataset(
        dir_data,
        valid_split_file,
        img_size=img_size,
        is_trainset=True,
        return_label=True,
        in_channels=in_channels,
        transform=None,
        crop_size=crop_size,
        random_crop=False)
    
    valid_loader = DataLoader(
        valid_dataset,
        sampler=SequentialSampler(valid_dataset),
        batch_size=batch_size,
        drop_last=False,
        num_workers=workers,
        pin_memory=pin_memory)

    focal_loss = FocalLoss().to(DEVICE)
    log.write('** start training here! **\n')
    log.write('\n')
    log.write('epoch    iter      rate     |  train_loss/acc  |    valid_loss/acc/focal/kaggle     |best_epoch/best_focal|  min \n')
    log.write('-----------------------------------------------------------------------------------------------------------------\n')
    start_epoch += 1
    for epoch in range(start_epoch, epochs + 1):
        end = time.time()

        # set manual seeds per epoch
        np.random.seed(epoch)
        torch.manual_seed(epoch)
        torch.cuda.manual_seed_all(epoch)

        # adjust learning rate for each epoch
        lr_list = scheduler.step(model, epoch, epochs)
        lr = lr_list[0]

        # train for one epoch on train set
        iter, train_loss, train_acc = train(train_loader, model, criterion, optimizer, epoch, clipnorm=clipnorm, lr=lr)

        with torch.no_grad():
            valid_loss, valid_acc, valid_focal_loss, kaggle_score = validate(valid_loader, model, criterion, epoch, focal_loss)

        # remember best loss and save checkpoint
        is_best = valid_focal_loss < best_focal
        best_loss = min(valid_focal_loss, best_loss)
        best_epoch = epoch if is_best else best_epoch
        best_focal = valid_focal_loss if is_best else best_focal

        print('\r', end='', flush=True)
        log.write('%5.1f   %5d    %0.6f   |  %0.4f  %0.4f  |    %0.4f  %6.4f %6.4f %6.4f    |  %6.1f    %6.4f   | %3.1f min \n' % \
                  (epoch, iter + 1, lr, train_loss, train_acc, valid_loss, valid_acc, valid_focal_loss, kaggle_score,
                   best_epoch, best_focal, (time.time() - end) / 60))

        save_model(model, is_best, model_out_dir, optimizer=optimizer, epoch=epoch, best_epoch=best_epoch, best_focal=best_focal)



def train(train_loader, model, criterion, optimizer, epoch, clipnorm=1, lr=1e-5):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    accuracy = AverageMeter()

    # switch to train mode
    model.train()

    num_its = len(train_loader)
    end = time.time()
    iter = 0
    print_freq = 1
    for iter, iter_data in enumerate(train_loader, 0):
        # measure data loading time
        data_time.update(time.time() - end)

        # zero out gradients so we can accumulate new ones over batches
        optimizer.zero_grad()

        images, labels, indices = iter_data
        images = Variable(images.to(DEVICE))
        labels = Variable(labels.to(DEVICE))

        outputs = model(images)
        loss = criterion(outputs, labels, epoch=epoch)

        losses.update(loss.item())
        loss.backward()

        torch.nn.utils.clip_grad_norm(model.parameters(), clipnorm)
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        logits = outputs
        probs = F.sigmoid(logits)
        acc = multi_class_acc(probs, labels)
        accuracy.update(acc.item())

        if (iter + 1) % print_freq == 0 or iter == 0 or (iter + 1) == num_its:
            print('\r%5.1f   %5d    %0.6f   |  %0.4f  %0.4f  | ... ' % \
                  (epoch - 1 + (iter + 1) / num_its, iter + 1, lr, losses.avg, accuracy.avg), \
                  end='', flush=True)

    return iter, losses.avg, accuracy.avg

def validate(valid_loader, model, criterion, epoch, focal_loss, threshold=0.5):
    batch_time = AverageMeter()
    losses = AverageMeter()
    accuracy = AverageMeter()

    # switch to evaluate mode
    model.eval()

    probs_list = []
    labels_list = []
    logits_list = []

    end = time.time()
    for it, iter_data in enumerate(valid_loader, 0):
        images, labels, indices = iter_data
        images = Variable(images.to(DEVICE))
        labels = Variable(labels.to(DEVICE))

        outputs = model(images)
        loss = criterion(outputs, labels, epoch=epoch)

        logits = outputs
        probs = F.sigmoid(logits)
        acc = multi_class_acc(probs, labels)

        probs_list.append(probs.cpu().detach().numpy())
        labels_list.append(labels.cpu().detach().numpy())
        logits_list.append(logits.cpu().detach().numpy())

        losses.update(loss.item())
        accuracy.update(acc.item())

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

    probs = np.vstack(probs_list)
    y_true = np.vstack(labels_list)
    logits = np.vstack(logits_list)
    valid_focal_loss = focal_loss.forward(torch.from_numpy(logits), torch.from_numpy(y_true))

    y_pred = probs > threshold
    kaggle_score = f1_score(y_true, y_pred, average='macro')

    return losses.avg, accuracy.avg, valid_focal_loss, kaggle_score

def save_model(model, is_best, model_out_dir, optimizer=None, epoch=None, best_epoch=None, best_focal=None):
    if type(model) == DataParallel:
        state_dict = model.module.state_dict()
    else:
        state_dict = model.state_dict()
    for key in state_dict.keys():
        state_dict[key] = state_dict[key].cpu()

    model_fpath = opj(model_out_dir, '%03d.pth' % epoch)
    torch.save({
        'save_dir': model_out_dir,
        'state_dict': state_dict,
        'best_epoch': best_epoch,
        'epoch': epoch,
        'best_score': best_focal,
    }, model_fpath)

    optim_fpath = opj(model_out_dir, '%03d_optim.pth' % epoch)
    if optimizer is not None:
        torch.save({
            'optimizer': optimizer.state_dict(),
        }, optim_fpath)

    if is_best:
        best_model_fpath = opj(model_out_dir, 'final.pth')
        shutil.copyfile(model_fpath, best_model_fpath)
        if optimizer is not None:
            best_optim_fpath = opj(model_out_dir, 'final_optim.pth')
            shutil.copyfile(optim_fpath, best_optim_fpath)

def multi_class_acc(preds, targs, th=0.5):
    preds = (preds > th).int()
    targs = targs.int()
    return (preds == targs).float().mean()

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0.
        self.avg = 0.
        self.sum = 0.
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
dir_results = Path('results')
dir_results.mkdir(exist_ok=True, parents=True)
out_dir = Path('external_crop256_focal_slov_hardlog_class_densenet121_dropout_i384_aug2_5folds')
model_multicell = (
    '../../kgl_humanprotein_data/result/models/'
    'external_crop512_focal_slov_hardlog_class_densenet121_dropout_i768_aug2_5folds/'
    'fold0/final.pth')
gpu_id = '0' # '0,1,2,3'
arch = 'class_densenet121_dropout'
num_classes = len(LABEL_NAME_LIST)
scheduler = 'Adam55'
epochs = 2 #55
resume = '001.pth'
sz_img = 384
crop_size = 64 #512
batch_size = 32
split_name = 'random_folds5'
fold = 0
workers = 0 # 3
pin_memory = False  # True

In [ ]:
main_training(dir_data, dir_mdata, dir_results, out_dir, 
              split_name=split_name, fold=fold,
              arch=arch, model_multicell=model_multicell, scheduler=scheduler,
              epochs=epochs, resume=resume,
              img_size=sz_img, crop_size=crop_size, batch_size=batch_size, 
              gpu_id=gpu_id, workers=workers, pin_memory=pin_memory)

>> Creating directory if it does not exist:
>> 'results/models/external_crop256_focal_slov_hardlog_class_densenet121_dropout_i384_aug2_5folds/fold0'
>> Using pre-trained model.
>> Loading multi-cell model.
>> Loading checkpoint:
>> 'results/models/external_crop256_focal_slov_hardlog_class_densenet121_dropout_i384_aug2_5folds/fold0/001.pth'
>> Loading checkpoint:
>> 'results/models/external_crop256_focal_slov_hardlog_class_densenet121_dropout_i384_aug2_5folds/fold0/001_optim.pth'
>>>> loaded checkpoint:
>>>> 'results/models/external_crop256_focal_slov_hardlog_class_densenet121_dropout_i384_aug2_5folds/fold0/001.pth' (epoch 1)
** start training here! **

epoch    iter      rate     |  train_loss/acc  |    valid_loss/acc/focal/kaggle     |best_epoch/best_focal|  min 
-----------------------------------------------------------------------------------------------------------------
  2.0       9    0.000300   |  38.5619  0.5519  |    14.4570  0.6470 22.1923 0.1003    |     2.0    22.1923   |

In [ ]:
! du -hs results/models/external_crop256_focal_slov_hardlog_class_densenet121_dropout_i384_aug2_5folds/fold0/

362M	results/models/external_crop256_focal_slov_hardlog_class_densenet121_dropout_i384_aug2_5folds/fold0/
